# Algorithmic Motion Planning (236610)
John Noonan and Eli Shafer<br>
Homework 2<br>
December 2019

### Warmup

Question 1: In a 2-dimensional unit ball (namely, a disk), how much of the volume is located at most 0.1 units from the surface?

**Answer: C Roughly 20%**

Question 2: In a 9-dimensional unit ball, how much of the volume is located at most 0.1 units from the surface?

**Answer: Roughly 60%**

### Exercise

Define the fraction of the volume that is $\epsilon$ distance from the surface of a d-dimensional unit ball as $n_d(\epsilon)$. 

**Plot** $\eta_d(\epsilon)$ as a function of $d$ for $\epsilon = 0.2, 0.1, 0.01$ for $d = 2 ... 10$

![plot_of_volume](ex_plot.png)

**Discuss** the implications to reducing the connection radius required for a sampling-based algorithm to maintain asymptotic optimality.

*This discussion is related to what we saw in class regarding the proof of probabilistic completeness for the PRM algorithm.  When proving asymptotic optimality for sampling-based algorithms, we typically take the path, cover it with balls where the radius of each ball is proportional to the clearance, and argue that within each ball, with high probability, we will get a sample and can connect two balls which will yield a free path since our connection radius is big enough.  It's important to note that the probabilistic argument depends on the volume of each one of the balls, and the notion of connection is critical because if the connection radius is too small, then there will be points in each ball but we **won't be able to connect them**.  In other words, if the connection radius is reduced too much then points will not be connected, nullifying our conclusions for asymptotic optimality.  On the other hand, if the connection radius is too large, then there will be too many points. Therefore, as we discussed, there has been a quest to determine the minimal connection radius or minimal expected number of nearest neighbors which are needed to determine certain desirable algorithmic properties.*

### 3 Motion Planning: Search and Sampling

#### 3.3 A* Implementation

A* was implemented and tested on the given map with `-s 450 0 -g 50 400`

#### Results
In the following, results are presented. In graphs unexplored space is in white, obstacles in black, path in red, explored space in colour with blue as the smallest $f(n)$ where 
$$f(n) = g(n) + h(n)$$
Where $g(n)$ is cost to node and $h(n)$ heuristic cost.

##### $\epsilon=1$
![](./img/astar_wf_e1v2.png)
expanded nodes: 18283
cost:  251.45079348883246


##### $\epsilon=10$
![](./img/astar_wf_e10v2.png)

expanded nodes: 1924

cost:  256.7228714274748


##### $\epsilon=20$
![](./img/astar_wf_e20v2.png)

expanded nodes: 2319

cost:  256.72287142747473


#### Discussion
It can be clearly seen in the results that the number of nodes expanded first decreases but then increases again with increasing $\epsilon$. With that comes the price of a less optimal path. It can also be seen with $\epsilon \gt 1$ that the wavefront opens up only once the algorithm encounters an obstacle.

#### RRT
RRT were done 10 times for each configuration:


##### Results for eta=5 and 5% goal sample probability:
cost avg:  316.2772647824529

cost stdev: 26.01204754785396

time avg 1.7362622976303101

time stdev: 1.560391613449739
![](img/rrt_eta5_gs5.png)

#### eta=5 and 20% goal sample probability:
cost avg:  281.04154222795796

cost stdev: 8.629722837915669

time avg 1.923483920097351

time stdev: 2.5501515976045384
![](img/rrt_eta5_gs20.png)

#### eta=inf and 5% goal sample probability:
cost avg:  483.5162513868522

cost stdev: 175.4014991263554

time avg 1.9762953281402589

time stdev: 3.1775145659089103
![](img/rrt_eta_inf_gs5.png)

#### eta=inf and 20% goal sample probability:
cost avg:  419.0239271273325

cost stdev: 76.96610786902902

time avg 2.4456452131271362

time stdev: 3.14332072198954
![](img/rrt_eta_inf_gs20.png)

#### RRT*
5 times were done `0.1, 0.5, 1.0, 5.0, 10.0` seconds. For each time there were 10 runs. Eta was set to 5.0 and goal sample was set to 5%.

For runtime of  0.1 secs
success:  0

For runtime of  0.5 secs
cost: 235.65589670679068
success:  1

For runtime of  1.0 secs
cost avg : 242.5866318945683
cost stdev: 10.929796988149397
success:  5

For runtime of  5.0 secs
cost avg : 244.5089908019645
cost stdev: 7.8413280880352785
success:  6

For runtime of  10.0 secs
cost avg : 239.07482823239076
cost stdev: 3.9489758020819106
success:  8
